 # Check if NVIDIA GPU is enabled 
 ( ensure that you enable "GPU" from Runtime -> Change Runtime Type )

In [1]:
!nvidia-smi

Sat Aug 15 21:22:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
 Bglore_Mysore_Trip		   'rev2 ppt (3)(1).gslides'
'Colab Notebooks'		   'SENIORS SPEAK'
'Google Photos'			   'Sweatshirt(neuronerdz).png'
'IEI STUDENT CHAPTER GOOGLE FORM'  'The Football Fever.gdoc'
'logo new (1).png'		    Travel_Grant_application.gdoc
'logo new.png'			   'tshirt layout(NeuroNerdz) (1).png'
'My Drive'			   'tshirt layout(NeuroNerdz).png'
 Project_Demo.mp4		    Video_Poster.mp4
 RA1511004010342.mpeg		    VQA_springer.gdoc
 report_1.gdoc			    

# Clone the darknet repo to get the codebase for YOLOV3

In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14240, done.
remote: Total 14240 (delta 0), reused 0 (delta 0), pack-reused 14240
Receiving objects: 100% (14240/14240), 12.78 MiB | 11.96 MiB/s, done.
Resolving deltas: 100% (9679/9679), done.


# Change makefile to have GPU and OPENCV enabled

In [4]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [6]:
!echo "racoon" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

# Download weights darknet model 53

In [7]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-08-15 21:25:29--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  1.17MB/s    in 2m 5s   

2020-08-15 21:27:35 (1.24 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



# Extract Images

#### The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [8]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
   creating: data/obj/images/
 extracting: data/obj/images/classes.txt  
  inflating: data/obj/images/raccoon-1.jpg  
 extracting: data/obj/images/raccoon-1.txt  
  inflating: data/obj/images/raccoon-10.jpg  
  inflating: data/obj/images/raccoon-10.txt  
  inflating: data/obj/images/raccoon-100.jpg  
  inflating: data/obj/images/raccoon-100.txt  
  inflating: data/obj/images/raccoon-101.jpg  
 extracting: data/obj/images/raccoon-101.txt  
  inflating: data/obj/images/raccoon-102.jpg  
 extracting: data/obj/images/raccoon-102.txt  
  inflating: data/obj/images/raccoon-103.jpg  
  inflating: data/obj/images/raccoon-103.txt  
  inflating: data/obj/images/raccoon-104.jpg  
  inflating: data/obj/images/raccoon-104.txt  
  inflating: data/obj/images/raccoon-105.jpg  
 extracting: data/obj/images/raccoon-105.txt  
  inflating: data/obj/images/raccoon-106.jpg  
  inflating: data/obj/images/raccoon-106.txt  
 extracting: data/obj/images/raccoon-107.jpg  
  i

# Converting the class index to 0 as we are doing single class object detection.

In [9]:
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/images/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.410625 0.456848 0.408750 0.891182

['15', '0.410625', '0.456848', '0.408750', '0.891182']
0 data/obj/images/raccoon-52.txt
0 0.410625 0.456848 0.408750 0.891182
15 0.422780 0.474227 0.351351 0.752577

['15', '0.422780', '0.474227', '0.351351', '0.752577']
1 data/obj/images/raccoon-198.txt
0 0.422780 0.474227 0.351351 0.752577
15 0.642857 0.463918 0.366795 0.577320

['15', '0.642857', '0.463918', '0.366795', '0.577320']
1 data/obj/images/raccoon-198.txt
0 0.642857 0.463918 0.366795 0.577320
15 0.683065 0.522581 0.569355 0.705376

['15', '0.683065', '0.522581', '0.569355', '0.705376']
2 data/obj/images/raccoon-85.txt
0 0.683065 0.522581 0.569355 0.705376
15 0.335938 0.550761 0.578125 0.827411

['15', '0.335938', '0.550761', '0.578125', '0.827411']
3 data/obj/images/raccoon-160.txt
0 0.335938 0.550761 0.578125 0.827411
15 0.551042 0.459722 0.352083 0.619444

['15', '0.551042', '0.459722', '0.352083', '0.619444']
4 data/obj/images/raccoon-40.txt
0 0.551042 0.459722 0.352083 0.619444
1

In [10]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(images_list)



['data/obj/images/raccoon-1.jpg', 'data/obj/images/raccoon-47.jpg', 'data/obj/images/raccoon-174.jpg', 'data/obj/images/raccoon-86.jpg', 'data/obj/images/raccoon-37.jpg', 'data/obj/images/raccoon-58.jpg', 'data/obj/images/raccoon-155.jpg', 'data/obj/images/raccoon-60.jpg', 'data/obj/images/raccoon-18.jpg', 'data/obj/images/raccoon-94.jpg', 'data/obj/images/raccoon-184.jpg', 'data/obj/images/raccoon-118.jpg', 'data/obj/images/raccoon-149.jpg', 'data/obj/images/raccoon-25.jpg', 'data/obj/images/raccoon-122.jpg', 'data/obj/images/raccoon-92.jpg', 'data/obj/images/raccoon-111.jpg', 'data/obj/images/raccoon-10.jpg', 'data/obj/images/raccoon-157.jpg', 'data/obj/images/raccoon-180.jpg', 'data/obj/images/raccoon-148.jpg', 'data/obj/images/raccoon-154.jpg', 'data/obj/images/raccoon-120.jpg', 'data/obj/images/raccoon-164.jpg', 'data/obj/images/raccoon-165.jpg', 'data/obj/images/raccoon-20.jpg', 'data/obj/images/raccoon-52.jpg', 'data/obj/images/raccoon-45.jpg', 'data/obj/images/raccoon-42.jpg', 

# Create training.txt file

In [11]:
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close()

# Start the training

In [12]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 85165, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.845839, GIOU: 0.843556), Class: 0.998145, Obj: 0.816448, No Obj: 0.002746, .5R: 1.000000, .75R: 1.000000, count: 4, class_loss = 0.363500, iou_loss = 0.099770, total_loss = 0.463270 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000006, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000002, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 85169, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.839997, GIOU: 0